In [19]:
from typing import TypedDict, Annotated, Sequence
from operator import add

class State(TypedDict):
  messages: Annotated[Sequence[str], add]

def function_1(state):
  return {"messages": ["uno"]}

def function_2(state):
  return {"messages": ["dos"]}

def function_3(state):
  return {"messages": ["tres"]}

In [20]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(State)
graph.add_node("node_1", function_1)
graph.add_node("node_2", function_2)
graph.add_node("node_3", function_3)
graph.add_edge(START, "node_1")
graph.add_edge("node_1", "node_2")
graph.add_edge("node_2", "node_3")
graph.add_edge("node_3", END)

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
app = graph.compile(checkpointer=memory)
user_input = {"messages": ["hola"]}
thread = {"configurable": {"thread_id": "1"}}
app.invoke(user_input, thread)

{'messages': ['hola', 'uno', 'dos', 'tres']}

In [22]:
for state in app.get_state_history(thread):
  print(state)
  print("--")

StateSnapshot(values={'messages': ['hola', 'uno', 'dos', 'tres']}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef6c516-4655-6fe6-8003-2635a073af23'}}, metadata={'source': 'loop', 'writes': {'node_3': {'messages': ['tres']}}, 'step': 3}, created_at='2024-09-06T13:10:30.429056+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef6c516-4651-6428-8002-69f5d744fea9'}})
--
StateSnapshot(values={'messages': ['hola', 'uno', 'dos']}, next=('node_3',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef6c516-4651-6428-8002-69f5d744fea9'}}, metadata={'source': 'loop', 'writes': {'node_2': {'messages': ['dos']}}, 'step': 2}, created_at='2024-09-06T13:10:30.427115+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef6c516-4649-64c6-8001-57cd5883bb73'}})
--
StateSnapshot(values={'messages': ['hola', 'uno']}, next=('node_2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef6c516-4649-64c6-8001-57cd5883bb73'}}, me